In [3]:
import tensorflow as tf
import data_util
import cv2
import matplotlib.pyplot as plt
import resnet_V2
from tensorflow.keras.utils import plot_model
from tensorflow import keras

## 데이터 불러오기

In [4]:
ds = data_util.get_cu_dataset(train_type='landmark')

In [5]:
ds_size = len(list(ds))
ds_size

239

In [6]:
train_size = int(0.7 * ds_size)
val_size = int(0.15 * ds_size)
test_size = int(0.15 * ds_size)
# full_dataset = tf.data.TFRecordDataset(FLAGS.input_file)
# ds = ds.shuffle()
train_ds = ds.take(train_size)
test_ds = ds.skip(train_size)
val_ds = test_ds.skip(val_size)
test_ds = test_ds.take(test_size)

In [7]:
len(list(train_ds))

167

In [16]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 256, 256, 3)
(32, 16)


## 모델 학습

In [7]:
en = resnet_V2.ResNetEncoder()

In [11]:
resnet = keras.models.Sequential()
resnet.add(keras.layers.Input(shape=(256, 256, 3)))
resnet.add(en)
resnet.add(keras.layers.Flatten())
resnet.add(keras.layers.Dense(256))
resnet.add(keras.layers.Dense(16))
resnet.compile(loss=keras.losses.mse,
              optimizer='adam')
              # metrics=['accuracy']   regression은 accuracy를 평가지표로 사용할 수 없음
              
# temp = tf.random.normal(shape=(1, 256, 256, 3))
# resnet(temp)
# resnet.summary()

In [9]:
train_ds

<TakeDataset shapes: ((None, None, None, 3), (None, None)), types: (tf.float32, tf.float32)>

In [12]:
# callbacks_list = [
#     # accuracy 기준 가장 높은 모델의 weight 저장
#     tf.keras.callbacks.ModelCheckpoint(
#         filepath = ckpt_path,
#         monitor='val_accuracy',
#         mode='max',
#         save_weights_only=True,
#         save_best_only=True
#     ),
#     # EarlyStopping
#     tf.keras.callbacks.EarlyStopping(
#         monitor='val_accuracy',
#         mode='max',
#         verbose=1, 
#         patience=20
#     )
# ]
resnet.compile(optimizer='adam', loss='mse')
hist = resnet.fit(train_ds, validation_data=val_ds, epochs=1)   # loss 기록

     51/Unknown - 582s 11s/step - loss: 1905.5244